In [18]:
using SimpleQP
using OSQP.MathOptInterfaceOSQP
using MathOptInterface
const MOI = MathOptInterface
using BenchmarkTools

function defaultoptimizer()
    optimizer = OSQPOptimizer()
    MOI.set!(optimizer, OSQPSettings.Verbose(), false)
    MOI.set!(optimizer, OSQPSettings.MaxIter(), 1000)
    MOI.set!(optimizer, OSQPSettings.WarmStart(), false)
    optimizer
end

function build_model(scale)
    model = Model(defaultoptimizer())
    p = let scale = scale
        Parameter(zeros(2), model) do v
            v[1] = 1.0
            v[2] = scale
        end
    end
    x = [Variable(model) for _ in 1:2]
    @constraint model x == p
    model, x
end

models = [build_model(scale) for scale in 1:4]

y = zeros(length(models))

function multi_solve!(y, models)
    let y = y, models = models
        Threads.@threads for i in 1:length(models)
            model, x = models[i]
            solve!(model)
            y[i] = SimpleQP.value(model, x[2])
        end
    end
    y
end

@btime multi_solve!($y, $models)


  22.170 μs (1 allocation: 48 bytes)


4-element Array{Float64,1}:
 0.999692
 1.99938 
 2.99908 
 3.99877 

In [15]:
@code_warntype multi_solve!(y, models)

Variables:
  #self# <optimized out>
  y@_2::Array{Float64,1}
  models@_3::Array{Tuple{SimpleQP.Model{Float64,OSQP.MathOptInterfaceOSQP.OSQPOptimizer},Array{SimpleQP.Functions.Variable,1}},1}
  range::UnitRange{Int64}
  threadsfor_fun::##35#threadsfor_fun#26{UnitRange{Int64},Array{Tuple{SimpleQP.Model{Float64,OSQP.MathOptInterfaceOSQP.OSQPOptimizer},Array{SimpleQP.Functions.Variable,1}},1},Array{Float64,1}}
  models@_6 <optimized out>
  y@_7 <optimized out>
  #temp#::Bool

Body:
  begin  # line 33:
      # meta: location threadingconstructs.jl # line 29:
      SSAValue(4) = (Base.arraylen)(models@_3::Array{Tuple{SimpleQP.Model{Float64,OSQP.MathOptInterfaceOSQP.OSQPOptimizer},Array{SimpleQP.Functions.Variable,1}},1})::Int64
      range::UnitRange{Int64} = $(Expr(:new, UnitRange{Int64}, 1, :((Base.select_value)((Base.sle_int)(1, SSAValue(4))::Bool, SSAValue(4), (Base.sub_int)(1, 1)::Int64)::Int64))) # line 30:
      threadsfor_fun::##35#threadsfor_fun#26{UnitRange{Int64},Array{Tuple{Simpl

In [ ]:
using BenchmarkTools

@everywhere function scalar_controller(scale)
    C = RemoteChannel{Channel{Float64}}
    input_channel::C = RemoteChannel(() -> Channel{Float64}(1), myid())
    output_channel::C = RemoteChannel(() -> Channel{Float64}(1), myid())
    controller = let scale = scale
        function (x)
            x * scale
        end
    end
    let input=input_channel, output=output_channel, controller=controller
        @async while true
            x = take!(input)
            y = controller(x)
            put!(output, y)
        end
    end
    input_channel, output_channel
end

channels = NTuple{2, RemoteChannel{Channel{Float64}}}[]

for i in 2:nprocs()
    push!(channels, @fetchfrom(i, scalar_controller(i)))
end

function scalar_parallel_control(channels, x)
    @sync begin
        for (input, output) in channels
            let input=input, output=output, x=x
                @async begin 
                    put!(input, x)
                    take!(output)
                end
            end
        end
    end
end